<a href="https://colab.research.google.com/github/EthanXC/RedditProject/blob/main/RedditProject1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

In [ ]:
print(dataset)

In [ ]:
it = iter(dataset['train'])


In [ ]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])
#testing if we can get a sample comment

In [ ]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [ ]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['text', 'score', 'labels'])

nsfw_keywords = ['nsfw', 'adult', 'sex', 'porn', 'xxx', 'gonewild', 'nudes', 'slut', 'WTF',
'hentai', 'cum', 'rule34', 'pussy', 'onlyfans', 'milf', 'nude', 'boob']

for i in range(10000):  # only 10,000 rows
    extract_data = next(it)
    text = extract_data['body']
    score = extract_data['score']
    labels = extract_data['subreddit']

    if any(keyword in labels.lower() for keyword in nsfw_keywords):
      continue

    new_row_data = {'text': text, "score": score, "labels": labels}
    df.loc[len(df)] = new_row_data

    if i % 1000 == 0:
        print(i)
        print(new_row_data)





In [ ]:
df.head()

In [ ]:
id2label = dict(enumerate(set(df['labels'])))

In [ ]:
id2label

In [ ]:
df.head()


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load DistilBERT model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = len(id2label))


In [ ]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

In [ ]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

In [ ]:
if np.isinf(df['score']).any():
  print("HELLO")

In [ ]:
print(df.dtypes)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [ ]:
id2label

In [ ]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [ ]:
id2label_reversed #word:number

In [ ]:
import torch as pt

class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            print(labels)
            labels_converted = id2label_reversed[labels]
            print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}

In [ ]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [ ]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [ ]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [ ]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
%set_env CUDA_LAUNCH_BLOCKING=1

In [ ]:
%env CUDA_LAUNCH_BLOCKING

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

In [ ]:
#set(id2label_reversed.values)

In [ ]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "I want to download minecraft mods"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")




In [ ]:
outputs.logits.shape

In [ ]:
len(id2label)

In [ ]:
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
print(probabilities)

In [ ]:
sort_prob = probabilities.sort(descending = True)

In [ ]:
print(sort_prob)

In [ ]:
def print_out_top_x_probs(probs, x):
  print('Here are the top 10 most subreddits with similar')
  for n in range(x):
    sort_probs = probs
    tensor_probs = sort_probs.values[n]
    index = sort_probs.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [ ]:
print_out_top_x_probs(sort_prob, 10)

In [ ]:
# it's time to graph!

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }


In [ ]:
import matplotlib.pyplot as plt

# this is fake data, replace with real data
epochs = 1
loss = [0.30, 0.24, 0.20]
f1_scores = [0.82, 0.85, 0.87]

plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plotting F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_scores, label='F1 Score', marker='o', color='r')
plt.title('F1 Score per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
